## Argsort()

Sort a list by their values and returned the indices of sorted items. 

In [3]:
def argsort(input):
    return sorted(range(len(input)), key=input.__getitem__)

A = [72,27,45,36,18,54,9,63]
print(argsort(A))
print([A[i] for i in argsort(A)])

print()
B = ["TAGACAT", "AGACAT", "GACAT", "ACAT", "CAT", "AT", "T"]
print(argsort(B))
print([B[i] for i in argsort(B)])

[6, 4, 1, 3, 2, 5, 7, 0]
[9, 18, 27, 36, 45, 54, 63, 72]

[3, 1, 5, 4, 2, 6, 0]
['ACAT', 'AGACAT', 'AT', 'CAT', 'GACAT', 'T', 'TAGACAT']


## Constructing a Suffix Array

In [1]:
def suffixArray(string):
    return [i for i in sorted(range(len(string)), key=lambda x: string[x:])]

t = "amanaplanacanalpanama$"
sa = suffixArray(t)
print(sa)
for i in sa:
    print("%2d: %s" % (i, t[i:]))

[21, 20, 9, 13, 18, 0, 7, 11, 16, 2, 4, 10, 6, 14, 19, 1, 8, 12, 17, 3, 15, 5]
21: $
20: a$
 9: acanalpanama$
13: alpanama$
18: ama$
 0: amanaplanacanalpanama$
 7: anacanalpanama$
11: analpanama$
16: anama$
 2: anaplanacanalpanama$
 4: aplanacanalpanama$
10: canalpanama$
 6: lanacanalpanama$
14: lpanama$
19: ma$
 1: manaplanacanalpanama$
 8: nacanalpanama$
12: nalpanama$
17: nama$
 3: naplanacanalpanama$
15: panama$
 5: planacanalpanama$


## Searching a Suffix Array

* Searching a sorted list requires $O(log(m))$ comparisons using ***binary search***
* Each comparision is over *n* symbols of the pattern
* Thus, searching is $O(n log(m))$

In [15]:
def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

pattern = "ana"
first = findFirst(pattern, t, sa)
print(t)
print(first, sa[first], t[sa[first]:])

amanaplanacanalpanama$
6 7 anacanalpanama$


## Finding all Occurences

A variant to binary search which finds the last occurence of a pattern rather than the first. Only difference, uses "<=" instead of "<", but needs to trim string comparison to test for equality.

In [16]:
def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

print(t)
last = findLast(pattern, t, sa)
print(first, last)
if last > first :
    for suffix in sa[first:last]:     # recall "first" was found on the previous slide
        print("%3d: %s" % (suffix, t[suffix:]))
    print(last - first, "matches of", pattern)
else:
    print("no match of", pattern)

amanaplanacanalpanama$
6 10
  7: anacanalpanama$
 11: analpanama$
 16: anama$
  2: anaplanacanalpanama$
4 matches of ana


## Longest repeated substring?

* Given a suffix array, we can compute a *helper* function, call the **Longest Common Prefix**, LCP

In [17]:
def computeLCP(text, suffixarray):
    m = len(text)
    lcp = [0 for i in range(m)]
    for i in range(1,m):
        u = suffixarray[i-1]
        v = suffixarray[i]
        n = 0
        while text[u] == text[v]:
            n += 1
            u += 1
            v += 1
            if (u >= m) or (v >= m):
                break
        lcp[i] = n
    return lcp

lcp = computeLCP(t, sa)

print("SA,LCP,Suffix")
for i, j in enumerate(sa):
    print("%2d: %2d %s" % (j, lcp[i], t[j:]))

SA,LCP,Suffix
21:  0 $
20:  0 a$
 9:  1 acanalpanama$
13:  1 alpanama$
18:  1 ama$
 0:  3 amanaplanacanalpanama$
 7:  1 anacanalpanama$
11:  3 analpanama$
16:  3 anama$
 2:  3 anaplanacanalpanama$
 4:  1 aplanacanalpanama$
10:  0 canalpanama$
 6:  0 lanacanalpanama$
14:  1 lpanama$
19:  0 ma$
 1:  2 manaplanacanalpanama$
 8:  0 nacanalpanama$
12:  2 nalpanama$
17:  2 nama$
 3:  2 naplanacanalpanama$
15:  0 panama$
 5:  1 planacanalpanama$


## Key Idea behind the BWT

* Sorting Cyclical Suffixes (say that 3-times fast)
<pre>
         "Cyclical Suffixes"         "Sorted Cyclical Suffixes" 
              tarheel$                    $tarhee<span style="color:blue;">l</span>
              arheel$t                    arheel$<span style="color:blue;">t</span>
              rheel$ta                    eel$tar<span style="color:blue;">h</span>
              heel$tar                    el$tarh<span style="color:blue;">e</span>
              eel$tarh                    heel$ta<span style="color:blue;">r</span>
              el$tarhe                    l$tarhe<span style="color:blue;">e</span>
              l$tarhee                    rheel$t<span style="color:blue;">a</span>
              $tarheel                    tarheel<span style="color:blue;">&dollar;</span>
</pre>

* The BWT of "tarheels" is the last column of the sorted cyclical suffixes "ltherea&dollar;"
* Notice that the sorted cyclical suffixes have a lot in common with a suffix array.
* The BWT is just the "predecessor symbol of these suffixes", where "&dollar;" precedes the first symbol

In [7]:
# making cyclical suffixes
t="carolina$"
print([t[i:]+t[:i] for i in range(len(t))])

['carolina$', 'arolina$c', 'rolina$ca', 'olina$car', 'lina$caro', 'ina$carol', 'na$caroli', 'a$carolin', '$carolina']


## BWT in Python
Straightforward implementation based on the definition (there are faster construction methods)

In [18]:
def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in range(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

print(BWT("banana$"))
print(BWT("amanaplanacanalpanama$"))
print(BWT("abananaban$"))
print(BWT("panamabananas$"))

annb$aa
amnnn$lcpmnapaaaaaaala
nn$bnbaaaaa
smnpbnnaaaaa$a


## BWT from a Suffix Array

* It is even simpler to compute the BWT from a Suffix Array
* Finds each suffix's "predecessor" symbol

In [19]:
def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)
    
newt = 'carolina$'
sa = suffixArray(newt)
print(newt)
print(sa)
print(BWTfromSuffixArray(newt, sa))

carolina$
[8, 7, 1, 0, 5, 4, 6, 3, 2]
anc$loira


## Inverse BWT in Python

In [22]:
def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in range(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

print(inverseBWT("ltherea$"))
print(inverseBWT("amnnn$lcpmnapaaaaaaala"))
print(inverseBWT("annb$aa"))
print(inverseBWT("nn$bnbaaaaa"))
print(inverseBWT("smnpbnnaaaaa$a"))

tarheel$
amanaplanacanalpanama$
banana$
abananaban$
panamabananas$


## Constructing the FM-index

In [48]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]  # a list of dictionaries
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].items()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

#bwt = "annb$aa"
bwt = "smnpbnnaaaaa$a"
FM, Offset = FMIndex(bwt)

print("BWT", [symbol for symbol in sorted(Offset.keys())])
for i, row in enumerate(FM):
    data = [bwt[i]+':' if i < len(bwt) else '  ']+[row[symbol] for symbol in sorted(row.keys())]
    print (data)
    
print()
print([(sym, Offset[sym]) for sym in sorted(Offset.keys())])

BWT ['$', 'a', 'b', 'm', 'n', 'p', 's']
['s:', 0, 0, 0, 0, 0, 0, 0]
['m:', 0, 0, 0, 0, 0, 0, 1]
['n:', 0, 0, 0, 1, 0, 0, 1]
['p:', 0, 0, 0, 1, 1, 0, 1]
['b:', 0, 0, 0, 1, 1, 1, 1]
['n:', 0, 0, 1, 1, 1, 1, 1]
['n:', 0, 0, 1, 1, 2, 1, 1]
['a:', 0, 0, 1, 1, 3, 1, 1]
['a:', 0, 1, 1, 1, 3, 1, 1]
['a:', 0, 2, 1, 1, 3, 1, 1]
['a:', 0, 3, 1, 1, 3, 1, 1]
['a:', 0, 4, 1, 1, 3, 1, 1]
['$:', 0, 5, 1, 1, 3, 1, 1]
['a:', 1, 5, 1, 1, 3, 1, 1]
['  ', 1, 6, 1, 1, 3, 1, 1]

[('$', 0), ('a', 1), ('b', 7), ('m', 8), ('n', 9), ('p', 12), ('s', 13)]


## Suffix Recovery

* What is the suffix array entry corresponding to BWT index *i*?
   - Start at *i* and repeatedly find predecessors until *i* is reached again

* To find the *original* string, just start with *i = 0*, the '$' index

In [49]:
def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

# recall that the FM-index that we built was "annb$aa", the BWT of "banana$"
for i in range(len(bwt)):
    print(i, recoverSuffix(i, bwt, FM, Offset), bwt[i])

0 $panamabananas s
1 abananas$panam m
2 amabananas$pan n
3 anamabananas$p p
4 ananas$panamab b
5 anas$panamaban n
6 as$panamabanan n
7 bananas$panama a
8 mabananas$pana a
9 namabananas$pa a
10 nanas$panamaba a
11 nas$panamabana a
12 panamabananas$ $
13 s$panamabanana a


## BWT string search in Python

* One of the simplest methods we've seen for searching

In [50]:
def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

print(findBWT("ana", FM, Offset))
print(findBWT("ban", FM, Offset))
print(findBWT("ann", FM, Offset))

(3, 6)
(7, 8)
(6, 6)
